In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt

# Import data

In [ ]:
#bids = pd.read_csv('data/bids.csv', sep=',')
#print('bids: OK {}'.format(bids.shape))
#bids.head()
bids = pd.read_csv('data/bids_normalized.csv', sep=',')
print('bids: OK {}'.format(bids.shape))
bids.head()

In [ ]:
bids_sorted = pd.read_csv('data/bids_sorted.csv', sep=',')
print('bids_sorted: OK {}'.format(bids_sorted.shape))
bids_sorted.head()

In [ ]:
#b_train = pd.read_csv('data/train.csv', sep=',')
#print('b_train: OK: {}'.format(b_train.shape))
#b_train.head()
b_train = pd.read_csv('data/b_train.csv', sep=',')
print('b_train: OK: {}'.format(b_train.shape))
b_train.head()

In [ ]:
#b_test = pd.read_csv('data/test.csv', sep=',')
#print('b_test: OK: {}'.format(b_test.shape))
#b_test.head()
b_test = pd.read_csv('data/b_test.csv', sep=',')
print('b_test: OK: {}'.format(b_test.shape))
b_test.head()

# PREPROCESSING

# bids_normalized table

In [ ]:
bids.head()

### 1. time_normalized

In [ ]:
min_time = bids['time'].min()
bids['time_normalized'] = round((bids['time'] - min_time) * 19 / 1e9,2)

In [ ]:
pd.DataFrame(bids.time_normalized.unique())
bids.head()

### 2. outcome

In [ ]:
bids['outcome'] = bids.merge(b_train, how='left', on='bidder_id').outcome

### other

In [ ]:
del bids['merchandise']

In [ ]:
del bids['device']
del bids['country']
del bids['ip']
del bids['url']

In [ ]:
bids.to_csv('data/bids_normalized_short.csv', sep=',', header=True, index=False)

# b_train table

In [ ]:
del b_train['address']
del b_train['payment_account']

### 1. mean_time_diff

In [ ]:
b_train['mean_time_diff'] = 0

In [ ]:
b_train.head()

In [ ]:
for bidder in b_train['bidder_id']:
    #d = bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)
    #print(d.mean())
    #b_train.loc[b_train[b_train.bidder_id == bidder].index,'mean_time_diff'] = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()
    
    m = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - 
         bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()[0]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'mean_time_diff'] = m
    
#d['shifted'] = d.shift(1)
#d['difference'] = d['time'] - d['shifted']
#d
    #plt.plot(d)
#plt.show()
#d

In [ ]:
b_train.loc[:,'mean_time_diff'] = round(b_train.mean_time_diff,5)

### 2. total_bids_count

In [ ]:
b_train['total_bids_count'] = 0

In [ ]:
b_train.head()

In [ ]:
i = 0
for bidder in b_train['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder].count()[0]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'total_bids_count'] = c

### 3. total_auctions_count

In [ ]:
b_train['total_auctions_count'] = 0

In [ ]:
b_train.head()

In [ ]:
i = 0
for bidder in b_train['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder]['auction'].nunique()
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'total_auctions_count'] = c

In [ ]:
b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)

### 4. mean_time_diff2

In [ ]:
b_train['mean_time_diff2'] = 0

In [ ]:
b_train.head()

In [ ]:
i = 0
for bidder in b_train['bidder_id'][:1]:
    if i % 10 == 0:
        print(i)
        print('bidder:', bidder)
        print(bids[bids.bidder_id == bidder].sort_values(by='time_normalized')[['auction','time_normalized']].count()[0])
    i += 1
    time_diffs = []
    for auction in bids[bids.bidder_id == bidder].sort_values(by='time_normalized')[['auction','time_normalized']].values:
        print(auction)
        df = bids[bids.auction == auction[0]].sort_values(by='time_normalized')[['bidder_id', 'time_normalized']]
        df['diff'] = bids[bids.auction == auction[0]].sort_values(by='time_normalized')['time_normalized'] - bids[bids.auction == auction[0]].sort_values(by='time_normalized')['time_normalized'].shift(1)
        c = np.array(df[(df.bidder_id == bidder) & (df.time_normalized == auction[1])]['diff'])[0]
        time_diffs.append(c)
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'mean_time_diff2'] = np.array(time_diffs).mean()

In [ ]:
bids_sorted = bids.sort_values(by=['auction', 'time'])
del bids_sorted['time']
del bids_sorted['outcome']
bids_sorted['diff'] = bids_sorted.time_normalized - bids_sorted.time_normalized.shift(1)

In [ ]:
i = 0
for auc in bids_sorted.auction.unique():
    print(i)
    i += 1
    bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index, 'diff2'] = bids_sorted[bids_sorted.auction == auc].time_normalized - bids_sorted[bids_sorted.auction == auc].time_normalized.shift(1)

In [ ]:
bids_sorted.to_csv('data/bids_sorted.csv', sep=',', header=True, index=False)

In [ ]:
i = 0
for auc in bids_sorted.auction.unique():
    if i % 200 == 0:
        print(i)
    i += 1
    bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index[0], 'diff'] = np.NaN
    #bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index, 'diff'] = bids_sorted[bids_sorted.auction == auc].time_normalized - bids_sorted[bids_sorted.auction == auc].time_normalized.shift(1)

In [ ]:
bids_sorted.head()

# 2) b_test table

In [27]:
del b_test['address']
del b_test['payment_account']

### 1. mean_time_diff

In [28]:
#b_test['mean_time_diff'] = 0

In [26]:
i = 0
#for bidder in b_test['bidder_id']:
    if i % 200 == 0:
        print(i)
    i += 1
    #d = bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)
    #print(d.mean())
    #b_test.loc[b_test[b_test.bidder_id == bidder].index,'mean_time_diff'] = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()
    
    m = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - 
         bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()[0]
    b_test.loc[b_test[b_test.bidder_id == bidder].index, 'mean_time_diff'] = m
    
#d['shifted'] = d.shift(1)
#d['difference'] = d['time'] - d['shifted']
#d
    #plt.plot(d)
#plt.show()
#d

In [10]:
b_test.loc[:,'mean_time_diff'] = round(b_test.mean_time_diff, 5)

### 2. total_bids_count

In [32]:
b_test['total_bids_count'] = 0

In [36]:
b_test.head()

,bidder_id,mean_time_diff,total_bids_count
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,444750.00000,4
1,a921612b85a1494456e74c09393ccb65ylp4y,722019.50000,3
2,6b601e72a4d264dab9ace9d7b229b47479v6i,345.62500,17
3,eaf0ed0afc9689779417274b4791726cn5udi,9890.55102,148
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,5678.22727,23


In [ ]:
i = 0
for bidder in b_test['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder].count()[0]
    b_test.loc[b_test[b_test.bidder_id == bidder].index, 'total_bids_count'] = c

### 3. total_auctions_count

In [37]:
b_test['total_auctions_count'] = 0

In [38]:
b_test.head()

,bidder_id,mean_time_diff,total_bids_count,total_auctions_count
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,444750.00000,4,0
1,a921612b85a1494456e74c09393ccb65ylp4y,722019.50000,3,0
2,6b601e72a4d264dab9ace9d7b229b47479v6i,345.62500,17,0
3,eaf0ed0afc9689779417274b4791726cn5udi,9890.55102,148,0
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,5678.22727,23,0


In [ ]:
i = 0
for bidder in b_test['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder]['auction'].nunique()
    b_test.loc[b_test[b_test.bidder_id == bidder].index, 'total_auctions_count'] = c

In [35]:
b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)